In [ ]:
# Install required packages (run this cell first if packages are not installed)
!pip install --user pandas gspread gspread-dataframe google-auth google-auth-oauthlib google-auth-httplib2

print("✅ All dependencies installed successfully!")
print("   You can now run the main code cell below.")

# 📊 Metabase to Google Sheets - Data Fetcher

**Self-contained notebook - Duplicate this file for each new sheet you want to create**

---

## ⚙️ Configuration

Edit the cell below to customize your data fetch:

## ⚠️ IMPORTANT: Google Sheets Limitations

**Google Sheets has a hard limit of 10 million cells per spreadsheet.**

Your current dataset: **2,004,530 rows × 17 columns = 34,077,112 cells**

This exceeds the limit by **3.4x**! Choose one solution below:

### 🔥 **Solution 1: Split into Multiple Sheets (Recommended)**
Split data by date ranges or categories into separate sheets within the same spreadsheet.

### 📊 **Solution 2: Use Multiple Spreadsheets**
Create separate spreadsheets for different time periods or data segments.

### 🗜️ **Solution 3: Reduce Data Size**
Filter data to only what's needed, or aggregate before uploading.

### 💾 **Solution 4: Use BigQuery or Database** 
For large datasets, consider using BigQuery directly instead of Google Sheets.

---

**Run the appropriate cell below based on your choice:**

### 🔥 SOLUTION 1: Split by Date Range (Recommended)

In [ ]:
# SOLUTION 1: Split Data by Month into Separate Sheets
# This uploads data in monthly chunks to different sheets in the SAME spreadsheet

import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
import sys
from datetime import datetime, timedelta
import time

# Configuration
SPREADSHEET_ID = "14XMNROBL6PT_GYq43AVJb9e_IowOZp_ZP_IpCwmQCgs"
SERVICE_ACCOUNT_FILE = "pw-service-22bdcc39f732.json"
DATE_COLUMN = "converteddate"  # Column to split by

# Add Trino utilities
sys.path.append('/home/jovyan/shared/python_utils/')
from python_utils_common import trino_prod

print("=" * 80)
print("📊 SPLIT UPLOAD TO GOOGLE SHEETS")
print("=" * 80)

# Your SQL Query (same as before)
SQL_QUERY = """
WITH main AS (
    SELECT *,
        COALESCE(NULLIF(TRIM(financeexamcategory), ''), exam) AS final_exam_category
    FROM cdp.mview.gold_dbt_orders_base_fact
),
pw_store_order_cte AS (
    SELECT
        batch_order_id AS ecom_id,
        SUM(net_price) AS add_on_store
    FROM cdp.mview.gold_pw_store_orders
    WHERE batch_order_id IS NOT NULL
    GROUP BY batch_order_id
),
plans AS (SELECT 
    batchid,
    name,
    startdate,
    CASE 
        WHEN pro_plan IS NOT NULL AND infinity_plan IS NOT NULL THEN 'Infinity_Pro_eligible'
        WHEN pro_plan IS NOT NULL AND infinity_plan IS NULL AND base_plan IS NULL THEN 'Only_pro_eligible'
        WHEN pro_plan IS NULL AND infinity_plan IS NOT NULL THEN 'Only_infinity_eligible'
        WHEN pro_plan IS NOT NULL AND infinity_plan IS NULL AND base_plan IS NOT NULL THEN 'base_pro_eligible'
        WHEN base_plan IS NOT NULL AND infinity_plan IS NULL AND pro_plan IS NULL THEN 'Only_base_batch'
        ELSE 'TBA'
    END AS batch_eligibility
FROM (
    SELECT 
        name,
        batchid,
        startdate,
        MAX(CASE WHEN displayorder = 1 AND type = 'BATCH' THEN _id END) AS base_plan,
        MAX(CASE WHEN type = 'INFINITY' THEN _id END) AS infinity_plan,
        MAX(CASE WHEN type = 'PRO' THEN _id END) AS pro_plan
    FROM (
        SELECT 
            b.exam,
            b.name,
            cast(b.startdate as date) as startdate,
            batchid,
            businessoffering,
            a.displayorder,
            a._id,
            CASE 
                WHEN lower(businessoffering) LIKE '%pro%' THEN 'PRO'
                WHEN lower(businessoffering) LIKE '%infinity%' THEN 'INFINITY'
                WHEN businessoffering = ''  THEN 'BATCH'
                ELSE 'BATCH'
            END AS type
        FROM cdp.cdp_revenue.gold_batch_plans a
        JOIN cdp.cdp_revenue.gold_batches_pw b
            ON a.batchid = b._id
        WHERE a.status = 'Active'
    ) inner_query
    GROUP BY 1, 2 ,3)),
filtration as (    
SELECT 
    m._id,
    m.batchid,
    m.plan,
    m.converteddate,
    m.name,
    m.net_amount,
    m.coupondiscount,
    m.couponcode,
    m.couponid,
    m.donationamount,
    m.Exam_2,
    m.order_type,
    p.batch_eligibility,
    p.startdate,
    s.ADD_ON_STORE,
    CASE 
    WHEN m.Exam_2 IN (
        'COMMERCE', 'CA', 'MBA', 'CLAT', 'IIT JAM', 'UGC NET', 'CS', 'CSIR NET',
        'Nursing Exams', 'IPMAT', 'CUET PG', 'Pharma', 'Design_Wallah', 'IELTS',
        'CA Final', 'ACCA', 'NEET_PG', 'CA Offline', 'MBA_GMAT', 'BFSI'
    ) THEN 'MANISH'
    
    WHEN m.Exam_2 IN (
        'SSC', 'Banking', 'WBPSC', 'TET', 'JAIIB AND CAIIB', 'State Exams'
    ) THEN 'PRASHANT'
    
    WHEN m.Exam_2 IN (
        'UPSC', 'BPSC', 'UPPSC', 'MPSC', 'Agriculture', 'MPPSC',
        'Judiciary', 'GATE', 'AE/JE', 'OPSC'
    ) THEN 'RAJAT'
    
    WHEN m.Exam_2 IN (
        'IIT-JEE', 'NEET', 'Foundation', 'Pre_Foundation', 'STATE BOARDS',
        'Defence', 'Vernacular', 'CUET UG', 'Bharat Batch'
    ) THEN 'SANYAM'
    
    WHEN m.Exam_2 IN ('Power Batch') THEN 'ANKIT'
    
    ELSE 'TBA'
END AS leader_fin
    
FROM (
    SELECT
        orderid AS _id,
        productid AS batchid,
        businessoffering AS plan,
        CAST(order_time AS DATE) AS converteddate,
        product_name AS name,
        order_price AS net_amount,
        order_coupondiscount AS coupondiscount,
        coupon_code AS couponcode,
        couponid,
        order_donationamount AS donationamount,
        CASE
            WHEN final_exam_category LIKE '%Vernacular%' THEN 'Vernacular'
            WHEN final_exam_category IN ('Bihar Exams', 'UP Exams', 'WB Exams') THEN 'State Exams'
            WHEN final_exam_category IN ('State PSC - WBPSC') THEN 'WBPSC'
            WHEN final_exam_category = 'Board_Exam'
                OR final_exam_category LIKE '%State%Board%' THEN 'STATE BOARDS'
            WHEN final_exam_category IN ('CUET Arts', 'CUET Science') THEN 'CUET UG'
            WHEN final_exam_category IN ('Commerce', 'CUET Commerce') THEN 'COMMERCE'
            WHEN final_exam_category IN ('ARCHITECTURE', 'Architecture', 'Design Wallah') THEN 'Design_Wallah'
            WHEN final_exam_category = 'State PSC - BPSC' THEN 'BPSC'
            WHEN final_exam_category = 'ESE' THEN 'GATE'
            WHEN final_exam_category = 'GMAT' THEN 'MBA_GMAT'
            WHEN final_exam_category = 'State PSC - MPPSC' THEN 'MPPSC'
            WHEN final_exam_category = 'State PSC - MPSC' THEN 'MPSC'
            WHEN final_exam_category = 'Nursing' THEN 'Nursing Exams'
            WHEN final_exam_category = 'State PSC - OPSC' THEN 'OPSC'
            WHEN final_exam_category = 'Railway' THEN 'SSC'
            WHEN final_exam_category = 'Teaching' THEN 'TET'
            WHEN final_exam_category = 'State PSC - UPPSC' THEN 'UPPSC'
            WHEN LOWER(final_exam_category) = 'foundation' THEN 'Foundation'
            WHEN final_exam_category IN ('Pre_Foundation', 'PRE_FOUNDATION') THEN 'Pre_Foundation'
            ELSE final_exam_category
        END AS exam_2,
        CASE 
            WHEN upgradedfrom IS NOT NULL THEN 'UPGRADE' 
            ELSE 'PRIMARY' 
        END AS order_type
    FROM main
    WHERE 
        isattribution = FALSE
        AND isprimary = TRUE
        AND issat = FALSE
        AND isvidyapeeth = FALSE
        AND ispathshala = FALSE
        AND (organizationid IN ('5eb393ee95fab7468a79d189', '63b52963e72e8b00186c11f3'))
        AND order_price > 0
        AND (type = 'BATCH' OR type = 'COMBO_PACKAGE')
        AND CAST(
                date_add('minute', 330, CAST(order_time AS timestamp))
            AS date
        ) BETWEEN DATE '2025-02-28' AND DATE '2025-11-06'
) m
LEFT JOIN pw_store_order_cte s
    ON m._id = s.ecom_id
LEFT JOIN plans p
    ON m.batchid = p.batchid)
SELECT *,
CASE 
    WHEN LOWER(plan) LIKE '%pro%' 
         AND Leader_FIN IN ('SANYAM', 'PRASHANT', 'MANISH', 'RAJAT') 
        THEN 'PRO'
        
    WHEN LOWER(plan) LIKE '%infinity%'
        THEN 'INFINITY'
        
    WHEN plan = '' 
        THEN 'BATCH'
        
    ELSE 'BATCH'
END AS type_2
FROM filtration
WHERE leader_fin = 'SANYAM'
AND Exam_2 IN ('IIT-JEE','NEET','Foundation','CUET UG')
"""

# Step 1: Fetch data
print("\n🔄 Fetching data from database...")
start_time = time.time()
df = trino_prod(SQL_QUERY)
print(f"✅ Fetched {len(df):,} rows in {time.time() - start_time:.2f}s")

# Step 2: Convert date column to datetime
df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN])

# Step 3: Split by month
df['year_month'] = df[DATE_COLUMN].dt.to_period('M')
grouped = df.groupby('year_month')

print(f"\n📊 Data will be split into {len(grouped)} monthly sheets:")
for period, group_df in grouped:
    cells = len(group_df) * len(group_df.columns)
    print(f"   - {period}: {len(group_df):,} rows = {cells:,} cells")

# Step 4: Upload each month to a separate sheet
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key(SPREADSHEET_ID)

print(f"\n⚡ Uploading to Google Sheets...")
total_uploaded = 0

for period, group_df in grouped:
    sheet_name = f"Batch_{period}"
    
    # Remove the temporary year_month column
    upload_df = group_df.drop('year_month', axis=1)
    
    rows = len(upload_df)
    cols = len(upload_df.columns)
    cells = (rows + 6) * cols  # +6 for metadata
    
    print(f"\n   📄 Sheet: {sheet_name}")
    print(f"      Rows: {rows:,} | Cells: {cells:,}")
    
    # Create or get worksheet
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
        print(f"      ✓ Using existing sheet")
    except:
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=rows+10, cols=cols)
        print(f"      ✓ Created new sheet")
    
    # Resize if needed
    if worksheet.row_count < rows + 10:
        worksheet.resize(rows=rows + 10, cols=cols)
    
    worksheet.clear()
    
    # Add metadata
    metadata = [
        ["Batch Enrollment Report"],
        [f"Period: {period}"],
        [f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"],
        [f"Records: {rows:,}"],
        []
    ]
    worksheet.update('A1:A5', metadata, value_input_option='RAW')
    
    # Upload data
    set_with_dataframe(worksheet, upload_df, row=6, include_index=False, include_column_header=True, resize=False)
    
    # Format header
    worksheet.format('A6:Q6', {
        "textFormat": {"bold": True},
        "backgroundColor": {"red": 0.2, "green": 0.5, "blue": 0.8}
    })
    
    total_uploaded += rows
    print(f"      ✅ Uploaded ({total_uploaded:,} / {len(df):,} total)")
    time.sleep(1)  # Rate limiting

print(f"\n" + "=" * 80)
print(f"✅ SUCCESS! Uploaded {total_uploaded:,} rows across {len(grouped)} sheets")
print(f"📊 View: https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/edit")
print("=" * 80)

### 🗜️ SOLUTION 2: Aggregate Data (Reduce Rows)

In [ ]:
# SOLUTION 2: Upload Only Aggregated Summary Data
# This reduces 2M rows to ~1000 rows by aggregating

import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
import sys
from datetime import datetime
import time

# Configuration
SPREADSHEET_ID = "14XMNROBL6PT_GYq43AVJb9e_IowOZp_ZP_IpCwmQCgs"
WORKSHEET_NAME = "Batch_Enrollment_Summary"
SERVICE_ACCOUNT_FILE = "pw-service-22bdcc39f732.json"

sys.path.append('/home/jovyan/shared/python_utils/')
from python_utils_common import trino_prod

print("=" * 80)
print("📊 AGGREGATED UPLOAD TO GOOGLE SHEETS")
print("=" * 80)

# Your SQL Query with aggregation - FIXED
SQL_QUERY_AGGREGATED = """
WITH main AS (
    SELECT *,
        COALESCE(NULLIF(TRIM(financeexamcategory), ''), exam) AS final_exam_category
    FROM cdp.mview.gold_dbt_orders_base_fact
),
pw_store_order_cte AS (
    SELECT
        batch_order_id AS ecom_id,
        SUM(net_price) AS add_on_store
    FROM cdp.mview.gold_pw_store_orders
    WHERE batch_order_id IS NOT NULL
    GROUP BY batch_order_id
),
plans AS (
    SELECT 
        batchid,
        name,
        startdate,
        CASE 
            WHEN pro_plan IS NOT NULL AND infinity_plan IS NOT NULL THEN 'Infinity_Pro_eligible'
            WHEN pro_plan IS NOT NULL AND infinity_plan IS NULL AND base_plan IS NULL THEN 'Only_pro_eligible'
            WHEN pro_plan IS NULL AND infinity_plan IS NOT NULL THEN 'Only_infinity_eligible'
            WHEN pro_plan IS NOT NULL AND infinity_plan IS NULL AND base_plan IS NOT NULL THEN 'base_pro_eligible'
            WHEN base_plan IS NOT NULL AND infinity_plan IS NULL AND pro_plan IS NULL THEN 'Only_base_batch'
            ELSE 'TBA'
        END AS batch_eligibility
    FROM (
        SELECT 
            name,
            batchid,
            startdate,
            MAX(CASE WHEN displayorder = 1 AND type = 'BATCH' THEN _id END) AS base_plan,
            MAX(CASE WHEN type = 'INFINITY' THEN _id END) AS infinity_plan,
            MAX(CASE WHEN type = 'PRO' THEN _id END) AS pro_plan
        FROM (
            SELECT 
                b.exam,
                b.name,
                cast(b.startdate as date) as startdate,
                batchid,
                businessoffering,
                a.displayorder,
                a._id,
                CASE 
                    WHEN lower(businessoffering) LIKE '%pro%' THEN 'PRO'
                    WHEN lower(businessoffering) LIKE '%infinity%' THEN 'INFINITY'
                    WHEN businessoffering = '' THEN 'BATCH'
                    ELSE 'BATCH'
                END AS type
            FROM cdp.cdp_revenue.gold_batch_plans a
            JOIN cdp.cdp_revenue.gold_batches_pw b ON a.batchid = b._id
            WHERE a.status = 'Active'
        ) inner_query
        GROUP BY 1, 2, 3
    )
),
base_data AS (
    SELECT
        productid AS batchid,
        businessoffering AS plan,
        CAST(order_time AS DATE) AS converteddate,
        product_name AS name,
        order_price AS net_amount,
        order_coupondiscount AS coupondiscount,
        order_donationamount AS donationamount,
        orderid,
        upgradedfrom,
        CASE
            WHEN final_exam_category LIKE '%Vernacular%' THEN 'Vernacular'
            WHEN final_exam_category IN ('Bihar Exams', 'UP Exams', 'WB Exams') THEN 'State Exams'
            WHEN final_exam_category IN ('State PSC - WBPSC') THEN 'WBPSC'
            WHEN final_exam_category = 'Board_Exam' OR final_exam_category LIKE '%State%Board%' THEN 'STATE BOARDS'
            WHEN final_exam_category IN ('CUET Arts', 'CUET Science') THEN 'CUET UG'
            WHEN final_exam_category IN ('Commerce', 'CUET Commerce') THEN 'COMMERCE'
            WHEN final_exam_category IN ('ARCHITECTURE', 'Architecture', 'Design Wallah') THEN 'Design_Wallah'
            WHEN final_exam_category = 'State PSC - BPSC' THEN 'BPSC'
            WHEN final_exam_category = 'ESE' THEN 'GATE'
            WHEN final_exam_category = 'GMAT' THEN 'MBA_GMAT'
            WHEN final_exam_category = 'State PSC - MPPSC' THEN 'MPPSC'
            WHEN final_exam_category = 'State PSC - MPSC' THEN 'MPSC'
            WHEN final_exam_category = 'Nursing' THEN 'Nursing Exams'
            WHEN final_exam_category = 'State PSC - OPSC' THEN 'OPSC'
            WHEN final_exam_category = 'Railway' THEN 'SSC'
            WHEN final_exam_category = 'Teaching' THEN 'TET'
            WHEN final_exam_category = 'State PSC - UPPSC' THEN 'UPPSC'
            WHEN LOWER(final_exam_category) = 'foundation' THEN 'Foundation'
            WHEN final_exam_category IN ('Pre_Foundation', 'PRE_FOUNDATION') THEN 'Pre_Foundation'
            ELSE final_exam_category
        END AS exam_2
    FROM main
    WHERE 
        isattribution = FALSE
        AND isprimary = TRUE
        AND issat = FALSE
        AND isvidyapeeth = FALSE
        AND ispathshala = FALSE
        AND (organizationid IN ('5eb393ee95fab7468a79d189', '63b52963e72e8b00186c11f3'))
        AND order_price > 0
        AND (type = 'BATCH' OR type = 'COMBO_PACKAGE')
        AND CAST(date_add('minute', 330, CAST(order_time AS timestamp)) AS date) 
            BETWEEN DATE '2025-02-28' AND DATE '2025-11-06'
),
filtration AS (    
    SELECT 
        bd.converteddate,
        bd.name,
        bd.exam_2,
        CASE 
            WHEN bd.upgradedfrom IS NOT NULL THEN 'UPGRADE' 
            ELSE 'PRIMARY' 
        END AS order_type,
        p.batch_eligibility,
        CASE 
            WHEN LOWER(bd.plan) LIKE '%pro%' 
                 AND bd.exam_2 IN ('IIT-JEE', 'NEET', 'Foundation', 'Pre_Foundation', 'STATE BOARDS', 'Defence', 'Vernacular', 'CUET UG', 'Bharat Batch')
                THEN 'PRO'
            WHEN LOWER(bd.plan) LIKE '%infinity%'
                THEN 'INFINITY'
            WHEN bd.plan = '' 
                THEN 'BATCH'
            ELSE 'BATCH'
        END AS type_2,
        bd.net_amount,
        bd.coupondiscount,
        bd.donationamount,
        s.ADD_ON_STORE,
        CASE 
            WHEN bd.exam_2 IN ('COMMERCE', 'CA', 'MBA', 'CLAT', 'IIT JAM', 'UGC NET', 'CS', 'CSIR NET', 'Nursing Exams', 'IPMAT', 'CUET PG', 'Pharma', 'Design_Wallah', 'IELTS', 'CA Final', 'ACCA', 'NEET_PG', 'CA Offline', 'MBA_GMAT', 'BFSI') THEN 'MANISH'
            WHEN bd.exam_2 IN ('SSC', 'Banking', 'WBPSC', 'TET', 'JAIIB AND CAIIB', 'State Exams') THEN 'PRASHANT'
            WHEN bd.exam_2 IN ('UPSC', 'BPSC', 'UPPSC', 'MPSC', 'Agriculture', 'MPPSC', 'Judiciary', 'GATE', 'AE/JE', 'OPSC') THEN 'RAJAT'
            WHEN bd.exam_2 IN ('IIT-JEE', 'NEET', 'Foundation', 'Pre_Foundation', 'STATE BOARDS', 'Defence', 'Vernacular', 'CUET UG', 'Bharat Batch') THEN 'SANYAM'
            WHEN bd.exam_2 IN ('Power Batch') THEN 'ANKIT'
            ELSE 'TBA'
        END AS leader_fin
    FROM base_data bd
    LEFT JOIN pw_store_order_cte s ON bd.orderid = s.ecom_id
    LEFT JOIN plans p ON bd.batchid = p.batchid
    WHERE 
        bd.exam_2 IN ('IIT-JEE','NEET','Foundation','CUET UG')
)
-- AGGREGATION: Group by date, batch, exam, type to reduce rows from 2M to ~thousands
SELECT
    converteddate,
    name AS batch_name,
    Exam_2,
    order_type,
    type_2 AS plan_type,
    batch_eligibility,
    leader_fin,
    COUNT(*) AS total_orders,
    SUM(net_amount) AS total_revenue,
    SUM(coupondiscount) AS total_discount,
    SUM(donationamount) AS total_donation,
    SUM(ADD_ON_STORE) AS total_addon_store,
    AVG(net_amount) AS avg_order_value
FROM filtration
WHERE leader_fin = 'SANYAM'
GROUP BY 1, 2, 3, 4, 5, 6, 7
ORDER BY 1 DESC, 2
"""

# Fetch aggregated data
print("\n🔄 Fetching aggregated data...")
start_time = time.time()
df = trino_prod(SQL_QUERY_AGGREGATED)
elapsed = time.time() - start_time

print(f"✅ Fetched {len(df):,} rows (aggregated) in {elapsed:.2f}s")
print(f"   Cells: {len(df) * len(df.columns):,} (fits easily!)")

# Upload to Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key(SPREADSHEET_ID)

try:
    worksheet = spreadsheet.worksheet(WORKSHEET_NAME)
except:
    worksheet = spreadsheet.add_worksheet(title=WORKSHEET_NAME, rows=len(df)+10, cols=len(df.columns))

worksheet.resize(rows=len(df)+10, cols=len(df.columns))
worksheet.clear()

metadata = [
    ["Batch Enrollment Report - Aggregated Summary"],
    [f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"],
    [f"Total Orders: {df['total_orders'].sum():,.0f}"],
    [f"Aggregated Rows: {len(df):,}"],
    []
]
worksheet.update('A1:A5', metadata, value_input_option='RAW')

set_with_dataframe(worksheet, df, row=6, include_index=False, include_column_header=True, resize=False)

print(f"\n✅ SUCCESS! Uploaded {len(df):,} aggregated rows")
print(f"📊 View: https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/edit#gid={worksheet.id}")
print("=" * 80)

ModuleNotFoundError: No module named 'python_utils_common'

In [ ]:
# Check available catalogs in Trino (run this once to find your catalog name)
import sys
sys.path.append('/home/jovyan/shared/python_utils/')
from python_utils_common import trino_prod

try:
    catalogs_df = trino_prod("SHOW CATALOGS")
    print("Available Catalogs in Trino:")
    print("=" * 50)
    display(catalogs_df)
    print("\nUse one of these catalog names in your query.")
    print("Example: catalog_name.cdp_revenue.gold_batches")
except Exception as e:
    print(f"Error: {e}")
    print("\nTrying alternative approach...")
    try:
        # Try to get current catalog
        session_df = trino_prod("SHOW SESSION")
        display(session_df)
    except Exception as e2:
        print(f"Error: {e2}")

In [ ]:
# ========================================================================================================
# CONFIGURATION - EDIT THIS SECTION
# ========================================================================================================

# Google Sheets Settings
SPREADSHEET_ID = "14XMNROBL6PT_GYq43AVJb9e_IowOZp_ZP_IpCwmQCgs"  # Your Google Sheet ID
WORKSHEET_NAME = "Batch Enrollment"  # Tab name in Google Sheet
SERVICE_ACCOUNT_FILE = "pw-service-22bdcc39f732.json"  # Service account JSON file

# Advanced Settings
AUTO_DELETE_OLD_SHEETS = True  # Automatically delete old sheets if space is needed
PROTECT_FIRST_SHEET = False  # Set to False to allow deleting the first sheet if needed for space

# Query Settings
QUERY_NAME = "Batch Enrollment Report"
SQL_QUERY = """
WITH main AS (
    SELECT *,
        COALESCE(NULLIF(TRIM(financeexamcategory), ''), exam) AS final_exam_category
    FROM cdp.mview.gold_dbt_orders_base_fact
),
pw_store_order_cte AS (
    SELECT
        batch_order_id AS ecom_id,
        SUM(net_price) AS add_on_store
    FROM cdp.mview.gold_pw_store_orders
    WHERE batch_order_id IS NOT NULL
    GROUP BY batch_order_id
),
plans AS (SELECT 
    batchid,
    name,
    startdate,
    CASE 
        WHEN pro_plan IS NOT NULL AND infinity_plan IS NOT NULL THEN 'Infinity_Pro_eligible'
        WHEN pro_plan IS NOT NULL AND infinity_plan IS NULL AND base_plan IS NULL THEN 'Only_pro_eligible'
        WHEN pro_plan IS NULL AND infinity_plan IS NOT NULL THEN 'Only_infinity_eligible'
        WHEN pro_plan IS NOT NULL AND infinity_plan IS NULL AND base_plan IS NOT NULL THEN 'base_pro_eligible'
        WHEN base_plan IS NOT NULL AND infinity_plan IS NULL AND pro_plan IS NULL THEN 'Only_base_batch'
        ELSE 'TBA'
    END AS batch_eligibility
FROM (
    SELECT 
        name,
        batchid,
        startdate,
        MAX(CASE WHEN displayorder = 1 AND type = 'BATCH' THEN _id END) AS base_plan,
        MAX(CASE WHEN type = 'INFINITY' THEN _id END) AS infinity_plan,
        MAX(CASE WHEN type = 'PRO' THEN _id END) AS pro_plan
    FROM (
        SELECT 
            b.exam,
            b.name,
            cast(b.startdate as date) as startdate,
            batchid,
            businessoffering,
            a.displayorder,
            a._id,
            CASE 
                WHEN lower(businessoffering) LIKE '%pro%' THEN 'PRO'
                WHEN lower(businessoffering) LIKE '%infinity%' THEN 'INFINITY'
                WHEN businessoffering = ''  THEN 'BATCH'
                ELSE 'BATCH'
            END AS type
        FROM cdp.cdp_revenue.gold_batch_plans a
        JOIN cdp.cdp_revenue.gold_batches_pw b
            ON a.batchid = b._id
        WHERE a.status = 'Active'
    ) inner_query
    GROUP BY 1, 2 ,3)),
filtration as (    
SELECT 
    m._id,
    m.batchid,
    m.plan,
    m.converteddate,
    m.name,
    m.net_amount,
    m.coupondiscount,
    m.couponcode,
    m.couponid,
    m.donationamount,
    m.Exam_2,
    m.order_type,
    p.batch_eligibility,
    p.startdate,
    s.ADD_ON_STORE,
    CASE 
    WHEN m.Exam_2 IN (
        'COMMERCE', 'CA', 'MBA', 'CLAT', 'IIT JAM', 'UGC NET', 'CS', 'CSIR NET',
        'Nursing Exams', 'IPMAT', 'CUET PG', 'Pharma', 'Design_Wallah', 'IELTS',
        'CA Final', 'ACCA', 'NEET_PG', 'CA Offline', 'MBA_GMAT', 'BFSI'
    ) THEN 'MANISH'
    
    WHEN m.Exam_2 IN (
        'SSC', 'Banking', 'WBPSC', 'TET', 'JAIIB AND CAIIB', 'State Exams'
    ) THEN 'PRASHANT'
    
    WHEN m.Exam_2 IN (
        'UPSC', 'BPSC', 'UPPSC', 'MPSC', 'Agriculture', 'MPPSC',
        'Judiciary', 'GATE', 'AE/JE', 'OPSC'
    ) THEN 'RAJAT'
    
    WHEN m.Exam_2 IN (
        'IIT-JEE', 'NEET', 'Foundation', 'Pre_Foundation', 'STATE BOARDS',
        'Defence', 'Vernacular', 'CUET UG', 'Bharat Batch'
    ) THEN 'SANYAM'
    
    WHEN m.Exam_2 IN ('Power Batch') THEN 'ANKIT'
    
    ELSE 'TBA'
END AS leader_fin
    
FROM (
    SELECT
        orderid AS _id,
        productid AS batchid,
        businessoffering AS plan,
        CAST(order_time AS DATE) AS converteddate,
        product_name AS name,
        order_price AS net_amount,
        order_coupondiscount AS coupondiscount,
        coupon_code AS couponcode,
        couponid,
        order_donationamount AS donationamount,
        CASE
            WHEN final_exam_category LIKE '%Vernacular%' THEN 'Vernacular'
            WHEN final_exam_category IN ('Bihar Exams', 'UP Exams', 'WB Exams') THEN 'State Exams'
            WHEN final_exam_category IN ('State PSC - WBPSC') THEN 'WBPSC'
            WHEN final_exam_category = 'Board_Exam'
                OR final_exam_category LIKE '%State%Board%' THEN 'STATE BOARDS'
            WHEN final_exam_category IN ('CUET Arts', 'CUET Science') THEN 'CUET UG'
            WHEN final_exam_category IN ('Commerce', 'CUET Commerce') THEN 'COMMERCE'
            WHEN final_exam_category IN ('ARCHITECTURE', 'Architecture', 'Design Wallah') THEN 'Design_Wallah'
            WHEN final_exam_category = 'State PSC - BPSC' THEN 'BPSC'
            WHEN final_exam_category = 'ESE' THEN 'GATE'
            WHEN final_exam_category = 'GMAT' THEN 'MBA_GMAT'
            WHEN final_exam_category = 'State PSC - MPPSC' THEN 'MPPSC'
            WHEN final_exam_category = 'State PSC - MPSC' THEN 'MPSC'
            WHEN final_exam_category = 'Nursing' THEN 'Nursing Exams'
            WHEN final_exam_category = 'State PSC - OPSC' THEN 'OPSC'
            WHEN final_exam_category = 'Railway' THEN 'SSC'
            WHEN final_exam_category = 'Teaching' THEN 'TET'
            WHEN final_exam_category = 'State PSC - UPPSC' THEN 'UPPSC'
            WHEN LOWER(final_exam_category) = 'foundation' THEN 'Foundation'
            WHEN final_exam_category IN ('Pre_Foundation', 'PRE_FOUNDATION') THEN 'Pre_Foundation'
            ELSE final_exam_category
        END AS exam_2,
        CASE 
            WHEN upgradedfrom IS NOT NULL THEN 'UPGRADE' 
            ELSE 'PRIMARY' 
        END AS order_type
    FROM main
    WHERE 
        isattribution = FALSE
        AND isprimary = TRUE
        AND issat = FALSE
        AND isvidyapeeth = FALSE
        AND ispathshala = FALSE
        AND (organizationid IN ('5eb393ee95fab7468a79d189', '63b52963e72e8b00186c11f3'))
        AND order_price > 0
        AND (type = 'BATCH' OR type = 'COMBO_PACKAGE')
        AND CAST(
                date_add('minute', 330, CAST(order_time AS timestamp))
            AS date
        ) BETWEEN DATE '2025-02-28' AND DATE '2025-11-06'
) m
LEFT JOIN pw_store_order_cte s
    ON m._id = s.ecom_id
LEFT JOIN plans p
    ON m.batchid = p.batchid)
SELECT *,
CASE 
    WHEN LOWER(plan) LIKE '%pro%' 
         AND Leader_FIN IN ('SANYAM', 'PRASHANT', 'MANISH', 'RAJAT') 
        THEN 'PRO'
        
    WHEN LOWER(plan) LIKE '%infinity%'
        THEN 'INFINITY'
        
    WHEN plan = '' 
        THEN 'BATCH'
        
    ELSE 'BATCH'
END AS type_2
FROM filtration
WHERE leader_fin = 'SANYAM'
AND Exam_2 IN ('IIT-JEE','NEET','Foundation','CUET UG')
"""

# Date Settings
DATE_MODE = 'yesterday'  # Options: 'today', 'yesterday', 'specific'
SPECIFIC_DATE = '2025-11-04'  # Used only if DATE_MODE = 'specific'
DAYS_BACK = 1  # Used only if DATE_MODE = 'yesterday'

# ========================================================================================================
# MAIN SCRIPT - NO NEED TO EDIT BELOW THIS LINE
# ========================================================================================================

print("=" * 80)
print("📊 METABASE TO GOOGLE SHEETS - DATA FETCHER")
print("=" * 80)

# Import libraries
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
import sys
from datetime import datetime, timedelta
import time

# Add Trino utilities path
sys.path.append('/home/jovyan/shared/python_utils/')
from python_utils_common import trino_prod

print("\n✅ Configuration loaded")
print(f"   Sheet: {WORKSHEET_NAME}")
print(f"   Date Mode: {DATE_MODE}")
print(f"   Query: {QUERY_NAME}")
print(f"   Auto-delete old sheets: {AUTO_DELETE_OLD_SHEETS}")
print(f"   Protect first sheet: {PROTECT_FIRST_SHEET}")

# ========================================================================================================
# Calculate Query Date
# ========================================================================================================

if DATE_MODE == 'today':
    query_date = datetime.now().date()
elif DATE_MODE == 'yesterday':
    query_date = (datetime.now() - timedelta(days=DAYS_BACK)).date()
elif DATE_MODE == 'specific':
    query_date = datetime.strptime(SPECIFIC_DATE, '%Y-%m-%d').date()
else:
    raise ValueError(f"Invalid DATE_MODE: {DATE_MODE}")

print(f"\n📅 Query Date: {query_date}")

# ========================================================================================================
# Execute Query and Fetch Data
# ========================================================================================================

formatted_query = SQL_QUERY.format(date=query_date)

print(f"\n🔄 Executing query: {QUERY_NAME}")
print(f"   Date: {query_date}")

try:
    start_time = time.time()
    df = trino_prod(formatted_query)
    elapsed = time.time() - start_time
    
    print(f"\n✅ Query completed in {elapsed:.2f} seconds")
    print(f"   Rows: {len(df)} | Columns: {len(df.columns)}")
    print(f"\n📊 Preview (first 3 rows):")
    display(df.head(3))
    
except Exception as e:
    print(f"\n❌ Error executing query: {str(e)}")
    raise

# ========================================================================================================
# Upload to Google Sheets (LIGHTNING FAST MODE WITH SMART SPACE MANAGEMENT)
# ========================================================================================================

if len(df) == 0:
    print("\n⚠️  No data to upload (query returned 0 rows)")
else:
    print(f"\n⚡ Uploading {len(df)} rows to Google Sheets...")
    upload_start = time.time()
    
    try:
        # Setup Google Sheets connection (cached credentials)
        scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
        creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=scope)
        client = gspread.authorize(creds)
        
        # Open spreadsheet
        spreadsheet = client.open_by_key(SPREADSHEET_ID)
        
        # Get or create worksheet
        try:
            worksheet = spreadsheet.worksheet(WORKSHEET_NAME)
            print(f"   ✓ Using existing worksheet: {WORKSHEET_NAME}")
        except gspread.exceptions.WorksheetNotFound:
            # Create with minimal initial size
            worksheet = spreadsheet.add_worksheet(title=WORKSHEET_NAME, rows=10, cols=5)
            print(f"   ✓ Created new worksheet: {WORKSHEET_NAME}")
        
        # Calculate required dimensions
        metadata_rows = 5
        required_rows = len(df) + metadata_rows + 1  # +1 for header
        required_cols = len(df.columns)
        required_cells = required_rows * required_cols
        
        print(f"   Required: {required_rows:,} rows x {required_cols} cols = {required_cells:,} cells")
        
        # Check current size
        current_rows = worksheet.row_count
        current_cols = worksheet.col_count
        current_sheet_cells = current_rows * current_cols
        
        # Calculate total cells across all sheets
        all_worksheets = spreadsheet.worksheets()
        total_cells_used = sum(ws.row_count * ws.col_count for ws in all_worksheets)
        cells_available = 10_000_000 - total_cells_used
        cells_needed_for_resize = required_cells - current_sheet_cells
        
        print(f"   Current spreadsheet: {total_cells_used:,} / 10,000,000 cells used")
        print(f"   Available: {cells_available:,} cells")
        print(f"   Need to add: {cells_needed_for_resize:,} cells")
        
        # Free up space if needed
        if cells_needed_for_resize > cells_available:
            cells_to_free = cells_needed_for_resize - cells_available + 100_000  # 100k buffer
            print(f"\n   ⚠️  Insufficient space! Need to free {cells_to_free:,} cells...")
            
            if not AUTO_DELETE_OLD_SHEETS:
                print("\n   ❌ AUTO_DELETE_OLD_SHEETS is set to False")
                print("   Options:")
                print("   1. Set AUTO_DELETE_OLD_SHEETS = True in configuration")
                print("   2. Manually delete old sheets from the spreadsheet")
                print(f"   3. View spreadsheet: https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/edit")
                raise Exception("Insufficient space and auto-delete is disabled")
            
            # Get all sheets sorted by size (largest first)
            worksheet_sizes = []
            for ws in all_worksheets:
                ws_cells = ws.row_count * ws.col_count
                ws_title = ws.title
                
                # Skip target worksheet
                if ws_title == WORKSHEET_NAME:
                    continue
                
                # Skip first sheet if protected
                if PROTECT_FIRST_SHEET and ws.id == all_worksheets[0].id:
                    continue
                
                worksheet_sizes.append((ws, ws_cells, ws_title))
            
            worksheet_sizes.sort(key=lambda x: x[1], reverse=True)
            
            # Calculate how many sheets we can delete
            total_deletable_cells = sum(size for _, size, _ in worksheet_sizes)
            
            if not worksheet_sizes:
                print("\n   ❌ No deletable sheets found!")
                print("   Current sheets:")
                for ws in all_worksheets:
                    protected = " [PROTECTED]" if (PROTECT_FIRST_SHEET and ws.id == all_worksheets[0].id) else ""
                    target = " [TARGET]" if ws.title == WORKSHEET_NAME else ""
                    print(f"      - {ws.title} ({ws.row_count * ws.col_count:,} cells){protected}{target}")
                print(f"\n   Options:")
                print(f"   1. Set PROTECT_FIRST_SHEET = False to allow deleting the first sheet")
                print(f"   2. Manually delete old sheets")
                print(f"   3. View spreadsheet: https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/edit")
                raise Exception("No deletable sheets available")
            
            if total_deletable_cells < cells_to_free:
                print(f"\n   ⚠️  WARNING: Can only free {total_deletable_cells:,} cells, but need {cells_to_free:,}")
                print(f"   This may still fail. Consider:")
                print(f"   1. Setting PROTECT_FIRST_SHEET = False")
                print(f"   2. Manually deleting more sheets")
                print(f"   3. Creating a new spreadsheet")
                print(f"\n   Attempting to delete all available sheets anyway...")
            
            cells_freed = 0
            sheets_deleted = 0
            
            for ws, size, title in worksheet_sizes:
                if cells_freed >= cells_to_free:
                    break
                
                print(f"   ✓ Deleting '{title}' ({size:,} cells)")
                spreadsheet.del_worksheet(ws)
                cells_freed += size
                sheets_deleted += 1
                time.sleep(0.5)  # Rate limiting
            
            print(f"   ✓ Freed {cells_freed:,} cells by deleting {sheets_deleted} sheet(s)")
            
            # Update available cells
            cells_available = cells_available + cells_freed
            if cells_needed_for_resize > cells_available:
                print(f"\n   ⚠️  Still insufficient space after deleting all available sheets")
                print(f"   Available: {cells_available:,} | Needed: {cells_needed_for_resize:,}")
                print(f"   Set PROTECT_FIRST_SHEET = False or manually delete more sheets")
                raise Exception("Insufficient space even after deleting all available sheets")
        
        # Dynamically resize ONLY if needed
        needs_resize = False
        new_rows = current_rows
        new_cols = current_cols
        
        if current_rows < required_rows:
            new_rows = required_rows
            needs_resize = True
        
        if current_cols < required_cols:
            new_cols = required_cols
            needs_resize = True
        
        if needs_resize:
            worksheet.resize(rows=new_rows, cols=new_cols)
            print(f"   ✓ Resized to {new_rows:,} rows x {new_cols} cols = {new_rows * new_cols:,} cells")
        
        # Clear existing data (FAST)
        worksheet.clear()
        
        # Prepare metadata
        metadata = [
            [QUERY_NAME],
            [f"Date: {query_date}"],
            [f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"],
            [f"Records: {len(df)}"],
            []
        ]
        
        # BATCH UPDATE - Update metadata in one call
        metadata_range = f'A1:A{len(metadata)}'
        worksheet.update(metadata_range, metadata, value_input_option='RAW')
        
        # Upload dataframe (FAST - single API call)
        set_with_dataframe(
            worksheet, 
            df, 
            row=len(metadata) + 1,
            include_index=False,
            include_column_header=True,
            resize=False  # We already resized, no need to resize again
        )
        
        # Format header (FAST - single API call)
        header_row = len(metadata) + 1
        worksheet.format(f'A{header_row}:{chr(65 + len(df.columns) - 1)}{header_row}', {
            "textFormat": {"bold": True},
            "backgroundColor": {"red": 0.2, "green": 0.5, "blue": 0.8}
        })
        
        upload_time = time.time() - upload_start
        print(f"\n✅ Upload completed in {upload_time:.2f} seconds")
        print(f"   Spreadsheet: https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/edit")
        print(f"   Worksheet: {WORKSHEET_NAME}")
        
    except Exception as e:
        print(f"\n❌ Error uploading to Google Sheets: {str(e)}")
        raise

# ========================================================================================================
# Summary
# ========================================================================================================

print("\n" + "=" * 80)
print("📋 EXECUTION SUMMARY")
print("=" * 80)
print(f"Query Name:    {QUERY_NAME}")
print(f"Query Date:    {query_date}")
print(f"Rows Fetched:  {len(df)}")
print(f"Columns:       {len(df.columns)}")
print(f"Sheet Tab:     {WORKSHEET_NAME}")
print(f"Column Names:  {', '.join(df.columns.tolist())}")
print("=" * 80)
print("\n🎉 Process completed successfully!")
print(f"View: https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/edit")
print("\n" + "=" * 80)